<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=359866e9aa3cfac5296b2ba73719361932972ded999d9c4fdea09fd5cd8c189f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 14:42:06
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: -67.89 K (-0.46%)
Current PnL: -18.63 L (-12.21%)
CY Booked + Current PnL: -6.50 L (-4.26%)
-------------------
Total profit:  2.57 L
Total loss:  -21.20 L
-------------------
Total Booked + Current PnL: 20.48 L (16.3%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.70 L (58.87%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.28,69.0,X-LC,4.55,232128.0,21762.0,7010.0,-1.08,10.34,3.02,13.68,37.0,3.10,1.61,28.58,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,106.83,59.0,M-SC,1.28,89152.0,-11625.0,11715.0,-2.29,-11.54,13.14,0.09,245.0,-0.99,0.62,17.13,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-16.77,71.0,H-MC,16.95,214243.0,34435.0,14911.0,-1.04,19.15,6.96,27.45,88.0,2.31,1.49,59.77,XR,NTT,BANKS
42,ITC,452.0,-36.66,62.0,X-LC,1.28,204891.0,4753.0,16145.0,-0.62,2.37,7.88,10.44,4.0,0.29,1.42,8.07,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.84,40.0,X-MC,2.50,89040.0,-1638.0,16544.0,-0.08,-1.81,18.58,16.44,101.0,-0.10,0.62,18.00,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,79.61,77.0,H-SC,2.67,155805.0,-26970.0,138043.0,7.22,-14.76,88.60,60.77,125.0,-0.20,1.08,39.55,XR,NTT,JEWELLERY
17,CAMS,5211.76,-2.50,57.0,X-SC,0.53,110774.0,8770.0,35160.0,2.61,8.60,31.74,43.06,122.0,0.25,0.77,27.18,X40N,ATH,MISC
65,SFL,1287.00,34.91,57.0,M-SC,5.97,178201.0,-84017.0,153841.0,1.90,-32.04,86.33,26.63,229.0,-0.55,1.24,10.80,XY24,NTT,MISC
50,MASFIN,398.61,-18.25,54.0,H-SC,9.17,92670.0,-5310.0,26911.0,1.81,-5.42,29.04,22.05,152.0,-0.20,0.64,34.70,XR,ATH,FINANCE
19,COALINDIA,497.97,7.46,50.0,L-LC,2.42,143652.0,-6808.0,40596.0,1.64,-4.53,28.26,22.46,167.0,-0.17,1.00,12.74,XY25,ATH,MINING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-31.78,52.0,H-SC,16.02,194797.0,-5157.0,102385.0,-3.44,-2.58,52.56,48.63,115.0,-0.05,1.35,8.68,XR,NTT,MISC
80,UNIONBANK,163.0,-10.08,54.0,M-LC,9.54,161386.0,20546.0,23950.0,-2.86,14.59,14.84,31.59,66.0,0.86,1.12,44.94,XY24,NTT,BANKS
62,SAIL,228.0,49.16,59.0,M-MC,10.23,235776.0,10814.0,156838.0,-2.58,4.81,66.52,74.53,192.0,0.07,1.64,38.61,XY24,BTT,STEEL
79,TTKPRESTIG,770.0,106.83,59.0,M-SC,1.28,89152.0,-11625.0,11715.0,-2.29,-11.54,13.14,0.09,245.0,-0.99,0.62,17.13,OX40N,NTT,DURABLES
54,QUESS,986.0,-46.01,48.0,X-SC,37.96,53819.0,-11187.0,162114.0,-2.21,-17.21,301.22,232.18,198.0,-0.07,0.37,4.98,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.0,-14.7,44.0,M-LC,5.69,149643.0,-1302.0,110751.0,-0.38,-0.86,74.01,72.51,53.0,-0.01,1.04,5.11,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.83,59.0,M-SC,1.28,89152.0,-11625.0,11715.0,-2.29,-11.54,13.14,0.09,245.0,-0.99,0.62,17.13,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,52.0,H-SC,2.83,225603.0,-44064.0,80405.0,-0.80,-16.34,35.64,13.47,138.0,-0.55,1.57,14.71,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,51.0,H-SC,2.49,140978.0,-34925.0,76946.0,0.05,-19.85,54.58,23.89,149.0,-0.45,0.98,21.75,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.53,53.0,H-MC,6.44,104794.0,-27041.0,69269.0,-0.42,-20.51,66.10,32.03,98.0,-0.39,0.73,17.38,OX40N,NTT,IT
68,SIS,528.0,2028.50,45.0,H-SC,3.21,85897.0,-25135.0,48747.0,-0.94,-22.64,56.75,21.26,156.0,-0.52,0.60,16.06,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,49.16,59.0,M-MC,10.23,235776.0,10814.0,156838.0,-2.58,4.81,66.52,74.53,192.0,0.07,1.64,38.61,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,56.0,M-LC,5.18,208038.0,4868.0,37260.0,-1.89,2.40,17.91,20.74,55.0,0.13,1.44,9.42,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.24,48.0,M-LC,11.32,207778.0,2702.0,109873.0,-1.23,1.32,52.88,54.89,52.0,0.02,1.44,25.73,X5K,ATH,METALS
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.45,132678.0,7381.0,51545.0,-0.98,5.89,38.85,47.03,141.0,0.14,0.92,66.13,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,54.0,M-LC,9.54,161386.0,20546.0,23950.0,-2.86,14.59,14.84,31.59,66.0,0.86,1.12,44.94,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-16.77,71.0,H-MC,16.95,214243.0,34435.0,14911.0,-1.04,19.15,6.96,27.45,88.0,2.31,1.49,59.77,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129886.0,15088.0,78958.0,0.04,13.14,60.79,81.92,223.0,0.19,0.90,53.76,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.45,132678.0,7381.0,51545.0,-0.98,5.89,38.85,47.03,141.0,0.14,0.92,66.13,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,57.0,H-SC,6.67,130804.0,7468.0,124159.0,-0.63,6.06,94.92,106.72,119.0,0.06,0.91,30.85,AR,ATH,MISC
32,HINDZINC,730.22,27.24,48.0,M-LC,11.32,207778.0,2702.0,109873.0,-1.23,1.32,52.88,54.89,52.0,0.02,1.44,25.73,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.04,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,88968.0,-39702.0,85000.0,-0.47,-30.86,95.54,35.20,219.0,-0.47,0.62,2.11,X40,NTT,FOOTWEAR
24,DMART,5391.45,-18.21,33.0,X-LC,9.67,100063.0,-2523.0,29328.0,-1.42,-2.46,29.31,26.13,38.0,-0.09,0.69,22.48,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.48,37.0,X-MC,5.24,199395.0,-952.0,31465.0,-0.27,-0.48,15.78,15.23,178.0,-0.03,1.38,31.62,X40N,NTT,REALTY
31,HINDUNILVR,2922.00,-11.98,38.0,X-LC,7.53,247190.0,-9873.0,45013.0,-0.65,-3.84,18.21,13.67,24.0,-0.22,1.72,15.79,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.83,46.0,H-LC,1.16,156010.0,-30085.0,77568.0,-0.91,-16.17,49.72,25.51,15.0,-0.39,1.08,14.97,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,62.0,X-LC,1.28,204891.0,4753.0,16145.0,-0.62,2.37,7.88,10.44,4.0,0.29,1.42,8.07,X40,NTT,FMCG
53,PGHH,17907.65,-30.68,38.0,X-MC,2.29,200790.0,-30.0,67827.0,-0.50,-0.01,33.78,33.76,80.0,-0.00,1.39,5.18,X40,ATH,FMCG
29,HAVELLS,2069.16,-4.90,52.0,X-MC,2.98,317247.0,-10744.0,117286.0,0.09,-3.28,36.97,32.48,92.0,-0.09,2.20,7.78,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,0.88,51.0,X-MC,4.19,211530.0,19788.0,17980.0,-0.61,10.32,8.50,19.69,99.0,1.10,1.47,17.82,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,88968.0,-39702.0,85000.0,-0.47,-30.86,95.54,35.20,219.0,-0.47,0.62,2.11,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.90,39.0,X-MC,6.70,81420.0,-2090.0,22122.0,-1.57,-2.50,27.17,23.98,82.0,-0.09,0.57,3.17,X40,ATH,APPARELS
3,ACC,3906.00,-37.20,52.0,X-MC,2.52,186130.0,-51821.0,204464.0,-0.96,-21.78,109.85,64.15,174.0,-0.25,1.29,4.41,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-13.84,50.0,X-MC,5.89,181384.0,-19122.0,116267.0,-0.05,-9.54,64.10,48.45,90.0,-0.16,1.26,4.60,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.23,50.0,X-MC,7.65,225780.0,-37585.0,150934.0,-0.44,-14.27,66.85,43.04,84.0,-0.25,1.57,4.82,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.01,48.0,X-SC,37.96,53819.0,-11187.0,162114.0,-2.21,-17.21,301.22,232.18,198.0,-0.07,0.37,4.98,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.90,39.0,X-MC,6.70,81420.0,-2090.0,22122.0,-1.57,-2.50,27.17,23.98,82.0,-0.09,0.57,3.17,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.10,47.0,X-SC,4.75,82955.0,-62205.0,138137.0,0.10,-42.85,166.52,52.31,136.0,-0.45,0.58,11.04,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.50,89040.0,-1638.0,16544.0,-0.08,-1.81,18.58,16.44,101.0,-0.10,0.62,18.00,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,88968.0,-39702.0,85000.0,-0.47,-30.86,95.54,35.20,219.0,-0.47,0.62,2.11,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.36,52.0,X-LC,10.94,285365.0,-60591.0,127301.0,-0.71,-17.51,44.61,19.28,1.0,-0.48,1.98,5.50,X40,ATH,IT
40,INFY,2275.00,-18.52,54.0,X-LC,5.80,317455.0,4567.0,167108.0,-1.32,1.46,52.64,54.87,3.0,0.03,2.20,9.86,X40,BTT,IT
42,ITC,452.00,-36.66,62.0,X-LC,1.28,204891.0,4753.0,16145.0,-0.62,2.37,7.88,10.44,4.0,0.29,1.42,8.07,X40,NTT,FMCG
84,VBL,671.64,-13.87,63.0,X-LC,8.47,310419.0,-5423.0,118083.0,-1.80,-1.72,38.04,35.67,5.0,-0.05,2.15,11.90,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,55.0,H-LC,5.60,252960.0,-8659.0,127871.0,-0.36,-3.31,50.55,45.57,7.0,-0.07,1.76,7.64,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6961.11,56.0,L-SC,7.80,77451.0,-16359.0,92972.0,-1.30,-17.44,120.04,81.67,269.0,-0.18,0.54,49.99,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.16,80678.0,-32871.0,72925.0,-0.66,-28.95,90.39,35.27,268.0,-0.45,0.56,97.46,XR,NTT,HOTELS
50,MASFIN,398.61,-18.25,54.0,H-SC,9.17,92670.0,-5310.0,26911.0,1.81,-5.42,29.04,22.05,152.0,-0.20,0.64,34.70,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129886.0,15088.0,78958.0,0.04,13.14,60.79,81.92,223.0,0.19,0.90,53.76,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.15,57.0,H-SC,6.67,130804.0,7468.0,124159.0,-0.63,6.06,94.92,106.72,119.0,0.06,0.91,30.85,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,79.61,77.0,H-SC,2.67,155805.0,-26970.0,138043.0,7.22,-14.76,88.60,60.77,125.0,-0.20,1.08,39.55,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.45,132678.0,7381.0,51545.0,-0.98,5.89,38.85,47.03,141.0,0.14,0.92,66.13,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129886.0,15088.0,78958.0,0.04,13.14,60.79,81.92,223.0,0.19,0.90,53.76,XR,NTT,DURABLES
5,ANGELONE,3033.00,16.33,64.0,X-SC,5.55,205820.0,14814.0,42893.0,-0.07,7.76,20.84,30.21,157.0,0.35,1.43,30.26,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.46,69.0,M-SC,7.82,84140.0,-17019.0,57022.0,-0.49,-16.82,67.77,39.54,202.0,-0.30,0.58,18.65,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.64
1,25,44.02
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.13
LC    32.28
MC    24.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.20
XY25     12.04
XR       11.47
X40N     10.69
AR        7.83
OX40N     6.55
X200      1.77
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.49
X-LC    21.63
X-MC    18.76
M-SC    12.67
M-LC     5.04
H-LC     4.61
X-SC     4.54
H-MC     3.87
M-MC     1.64
L-SC     1.43
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.53,-2.92,36.50
IT,12.47,-17.04,79.82
FINANCE,12.13,-8.06,57.04
MISC,6.85,-16.46,80.99
ELECTRICAL,5.81,-7.93,46.19
BANKS,5.62,-5.72,60.39
PAINTS,5.51,-16.35,33.70
INSURANCE,3.78,-1.45,39.27
AUTO,2.76,-44.50,104.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3247959.0,22
XR,1234043.0,14
AR,1160255.0,9
X40,821701.0,12
X40N,618216.0,9
OX40N,541252.0,9
XY25,449425.0,8
SR,267474.0,2
X5K,109873.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3302923.0,24
M-SC,1218153.0,15
X-MC,1175333.0,14
X-LC,1020553.0,13
X-SC,527319.0,6
M-LC,281834.0,4
H-LC,273914.0,3
H-MC,260363.0,3
L-SC,255406.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1152965.0      6
           AR         831123.0      5
           XR         759016.0      7
M-SC       XY24       698376.0      6
X-MC       XY24       573955.0      4
X-LC       X40        451969.0      5
           XY24       303578.0      2
X-MC       X40        284732.0      6
X-SC       X40N       280205.0      4
H-SC       SR         267474.0      2
           OX40N      240800.0      3
H-LC       AR         205439.0      2
X-LC       X40N       190279.0      3
H-MC       XY24       176183.0      1
X-MC       XY25       168914.0      2
L-SC       XR         165897.0      2
X-SC       XY24       162114.0      1
M-MC       XY24       156838.0      1
X-MC       X40N       147732.0      2
M-SC       OX40N      141674.0      4
           XY25       127928.0      1
           XR         126482.0      2
           AR         123693.0      2
M-LC       XR         110751.0      1
           X5K        109873.0      1
L-SC       OX40N       89509.0      1
X-SC       X40         85000.0      1
X-LC       XY25        74727.0      3
H-MC       OX40N       69269.0      1
H-LC       X200        68475.0      1
L-MC       XR          56986.0      1
H-SC       MH          51545.0      1
L-LC       XY25        40596.0      1
M-LC       XY25        37260.0      1
           XY24        23950.0      1
H-MC       XR          14911.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
